In [1]:
import numpy as np
import pandas as pd
from sklearn import cross_validation, tree, metrics
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble.weight_boosting import AdaBoostClassifier
from sklearn.externals.six.moves import xrange
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
import seaborn as sns
%pylab inline
from sklearn import cross_validation, datasets, svm
from sklearn.cross_validation import KFold, train_test_split
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier

Populating the interactive namespace from numpy and matplotlib


In [2]:
dataset = pd.read_csv('proc_data.csv')
dataset['id'] = dataset['id'].astype(int)

In [3]:
train = dataset.query('type == "train"')         
test = dataset.query('type == "test"')  

In [4]:
features = ['age', 'class', 'fare', 'family_size', 'embarked_at_Q', 'embarked_at_S', 'female', 'in_cabin', 'Master', 'Miss', 'Mrs']
X = train[['age', 'class', 'fare',  'family_size', 'embarked_at_Q', 'embarked_at_S', 'female', 'in_cabin', 'Master', 'Miss', 'Mrs']].values
y = train['survived'].values

In [5]:
X_test = test[['age', 'class', 'fare', 'family_size', 'embarked_at_Q', 'embarked_at_S', 'female', 'in_cabin', 'Master', 'Miss', 'Mrs']].values
test_ids = test['id']
test_ids = pd.DataFrame(test_ids.reset_index()['id'])

**Random Forest Confusion**

In [25]:
rnd = RandomForestClassifier(n_estimators = 1000, max_depth = 5, oob_score=True)
rnd.fit(X, y)
y_predicted = rnd.predict(X)
print accuracy_score(y, y_predicted)
print rnd.oob_score_

0.855218855219
0.827160493827


In [7]:
cm = confusion_matrix(y, y_predicted)
print(cm)

[[515  34]
 [ 93 249]]


In [8]:
g = pd.DataFrame(y_predicted)
g.columns = ['pred']

In [9]:
z = pd.concat([train, g], axis = 1)

In [10]:
def misClass(row):
    if row['survived'] != row['pred']:
        return 1
    else:
        return 0

z['misclassified'] = z.apply(misClass, axis = 1)

In [11]:
z

,age,cabin,class,embarked,fare,id,name,parch,sex,sibsp,...,family_id,family_id_int,Master,Miss,Mr,Mrs,Rev,farebin,pred,misclassified
0,22,NaN,3,S,7.2500,1,"Braund, Mr. Owen Harris",0,male,1,...,Braund-2.0,105,0,0,1,0,0,1,0,0
1,38,C85,1,C,71.2833,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,female,1,...,Cumings-2.0,190,0,0,0,1,0,14,1,0
2,26,NaN,3,S,7.9250,3,"Heikkinen, Miss. Laina",0,female,0,...,Heikkinen-1.0,352,0,1,0,0,0,1,0,1
3,35,C123,1,S,53.1000,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,female,1,...,Futrelle-2.0,281,0,0,0,1,0,10,1,0
4,35,NaN,3,S,8.0500,5,"Allen, Mr. William Henry",0,male,0,...,Allen-1.0,15,0,0,1,0,0,1,0,0
5,26,NaN,3,Q,8.4583,6,"Moran, Mr. James",0,male,0,...,Moran-1.0,573,0,0,1,0,0,1,0,0
6,54,E46,1,S,51.8625,7,"McCarthy, Mr. Timothy J",0,male,0,...,McCarthy-1.0,533,0,0,1,0,0,10,0,0
7,2,NaN,3,S,21.0750,8,"Palsson, Master. Gosta Leonard",1,male,3,...,Palsson-5.0,648,1,0,0,0,0,4,0,0
8,27,NaN,3,S,11.1333,9,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",2,female,0,...,Johnson-3.0,413,0,0,0,1,0,2,1,0
9,14,NaN,2,C,30.0708,10,"Nasser, Mrs. Nicholas (Adele Achem)",0,female,1,...,Nasser-2.0,596,0,0,0,1,0,6,1,0


In [12]:
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

In [13]:
query = '''
select
sex,
title,
class,
sum(misclassified) / count(1) as pct_mis,
count(1) as count,
sum(survived) / count(1) as surv_rate,
sum(pred) / count(1) as pred_surv_rate
from z
group by sex, title, class
'''
q = pysqldf(query)

**Exploration of Actual vs Predicted Survival Rates**

In [14]:
q

,sex,title,class,pct_mis,count,surv_rate,pred_surv_rate
0,female,Miss,1,0.043478,46,0.956522,1.000000
1,female,Miss,2,0.058824,34,0.941176,1.000000
2,female,Miss,3,0.205882,102,0.500000,0.588235
3,female,Mr,1,0.000000,1,1.000000,1.000000
4,female,Mrs,1,0.021277,47,0.978723,1.000000
5,female,Mrs,2,0.095238,42,0.904762,1.000000
6,female,Mrs,3,0.261905,42,0.500000,0.714286
7,male,Master,1,0.000000,3,1.000000,1.000000
8,male,Master,2,0.000000,9,1.000000,1.000000
9,male,Master,3,0.035714,28,0.392857,0.357143


**Attempt to over sample 3rd Class women who are consistently classified optimistically**

In [15]:
over_sample = '''
select*
from train
where class = 3 and sex = 'female'
'''
xtra = pysqldf(over_sample)

/Users/jb/anaconda/lib/python2.7/site-packages/pandasql/sqldf.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df[col] = df[col].astype(np.float)


In [16]:
train_xtra = pd.concat([train, xtra])

In [17]:
train_xtra

,age,cabin,class,embarked,fare,id,name,parch,sex,sibsp,...,in_cabin,family_size,family_id,family_id_int,Master,Miss,Mr,Mrs,Rev,farebin
0,22.0,NaN,3,S,7.2500,1,"Braund, Mr. Owen Harris",0,male,1,...,0,2,Braund-2.0,105,0,0,1,0,0,1
1,38.0,C85,1,C,71.2833,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,female,1,...,1,2,Cumings-2.0,190,0,0,0,1,0,14
2,26.0,NaN,3,S,7.9250,3,"Heikkinen, Miss. Laina",0,female,0,...,0,1,Heikkinen-1.0,352,0,1,0,0,0,1
3,35.0,C123,1,S,53.1000,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,female,1,...,1,2,Futrelle-2.0,281,0,0,0,1,0,10
4,35.0,NaN,3,S,8.0500,5,"Allen, Mr. William Henry",0,male,0,...,0,1,Allen-1.0,15,0,0,1,0,0,1
5,26.0,NaN,3,Q,8.4583,6,"Moran, Mr. James",0,male,0,...,0,1,Moran-1.0,573,0,0,1,0,0,1
6,54.0,E46,1,S,51.8625,7,"McCarthy, Mr. Timothy J",0,male,0,...,1,1,McCarthy-1.0,533,0,0,1,0,0,10
7,2.0,NaN,3,S,21.0750,8,"Palsson, Master. Gosta Leonard",1,male,3,...,0,5,Palsson-5.0,648,1,0,0,0,0,4
8,27.0,NaN,3,S,11.1333,9,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",2,female,0,...,0,3,Johnson-3.0,413,0,0,0,1,0,2
9,14.0,NaN,2,C,30.0708,10,"Nasser, Mrs. Nicholas (Adele Achem)",0,female,1,...,0,2,Nasser-2.0,596,0,0,0,1,0,6


In [18]:
X_xtra = train[['age', 'class', 'fare',  'family_size', 'embarked_at_Q', 'embarked_at_S', 'female', 'in_cabin', 'Master', 'Miss', 'Mrs']].values
y_xtra = train['survived'].values

In [19]:
rnd = RandomForestClassifier(n_estimators = 1000, max_depth = 10, oob_score=True)
rnd.fit(X_xtra, y_xtra)
y_predicted = rnd.predict(X_xtra)
print accuracy_score(y_xtra, y_predicted)
print rnd.oob_score_

0.945005611672
0.826038159371


In [20]:
rnd = RandomForestClassifier(n_estimators = 1000, max_depth = 10, oob_score=True)
rnd.fit(X_xtra, y_xtra)
y_test_predicted = rnd.predict(X_test)
predictions = pd.DataFrame(y_test_predicted)
submission = pd.concat([test_ids, predictions], axis=1)
submission.columns = ['PassengerId', 'Survived']
submission['Survived'] = submission['Survived'].astype(int)
submission.to_csv('rf_oversampled.csv', sep = ',', index = False)

**Look at using a more complex model just for 3rd Class Females**

Base Model

In [29]:
rnd = RandomForestClassifier(n_estimators = 1000, max_depth = 5, oob_score=True)
rnd.fit(X, y)
y_test_predicted = rnd.predict(X_test)
base_predictions = pd.DataFrame(y_test_predicted)
base_predictions.columns = ['base']

More Complex Model

In [30]:
rnd = RandomForestClassifier(n_estimators = 1000, max_depth = 10, oob_score=True)
rnd.fit(X, y)
y_test_predicted = rnd.predict(X_test)
alt_predictions = pd.DataFrame(y_test_predicted)
alt_predictions.columns = ['alt']

In [75]:
base_predictions.index += 891
alt_predictions.index += 891

In [76]:
new_preds = pd.concat([test, base_predictions, alt_predictions], axis = 1)

In [81]:
new_preds = new_preds[['sex', 'title', 'class', 'base', 'alt']]
new_preds.head()

,sex,title,class,base,alt
891,male,Mr,3,0,0
892,female,Mrs,3,0,0
893,male,Mr,2,0,0
894,male,Mr,3,0,0
895,female,Mrs,3,1,1


In [87]:
def altPred(row):
    if row['sex'] == 'female' and row['class'] == 3:
        return row['alt']
    else:
        return row['base']

new_preds['prediction'] = z.apply(altPred, axis = 1)

KeyError: ('base', u'occurred at index 0')

In [88]:
new_preds.to_csv('new_preds', sep = ',', index = False)